# 9 Acoustic Modeling

对于给定的声学特征 \\(\mathbf{X}=X_1 X_2 ... X_n\\)，语音识别的目标是需要一个字序列 \\(\mathbf{\hat{W}}={\omega}_1 {\omega}_2 ... {\omega}_m\\) 使得它的后验概率 \\(P(\mathbf{W}|\mathbf{X})\\) 最大，即

$$\hat{W}=\mathop{\arg\max}_{\omega}P(\mathbf{W}|\mathbf{X})=\mathop{\arg\max}_{\omega}\frac{P(\mathbf{W})P(\mathbf{X}|\mathbf{W})}{P(\mathbf{X})}$$

其中\\(P(X|W)\\)由声学模型得到，\\(P(W)\\)由语言模型得到。

## 9.1 Variability in the Speech Signal
在声学模型层面，影响识别率的因素有：
* Context Variability
    1. Word 层面相同发音的词活短语。e.g. Mr. **Wright** should **write** to Ms. **Wright right** away about his **Ford** or **four door** Honda.
    2. Phoneme 层面。Phoneme的发音与上下文有关。e.g. Phoneme /ee/ 在peat 和 wheel 的发音也不一样。
* Style Variability。 
    1. 口音，语速等
* Speaker Variability。 
    1. 说话人差异，对同样的词，每个人的发音也不一样。
    2. 同样的人，在不同的时间段、或不同的环境下，发音也会不一样。
* Environment Variability。 
    1. 环境差异，不同的环境，背景噪声数据不一样。

## 9.2 How to Measure Speech Recognition Errors
语音识别性能评估的常用指标：**Word error rate**. 它又可细分为三种情况：
* Substitution: an incorrect word was substituted for the correct word
* Deletion: a correct word was omitted in the recognized sentence
* Insertion: an extra word was added in the recognized sentence

$$Word Error Rate=100\% \times \frac{Subs + Dels + Ins}{No.\ of\ word\ in\ the\ correct\ sentence}$$

Subs, Dels 和 Ins 的计算其实就是求解**最大字符子串**的问题，可通过DP（动态规划）算法来计算，算法如下图：
![图9-1](./images/9_1.png)
对于反馈 拒绝/接受 结果的应用场景，e.g. speaker or command verification, wakeup, keywords spotting. 常采用FA-FR曲线（ROC,  receiver operating characteristic）来评价性能：
 * FR: False rejection
 * FA: False acceptance

## 9.3 Signal Processing - Extracting Feature
特征提取的目的：
* 减小数据量
* 移除噪声
* 提取语音信号特征

### 9.3.1 信号采样
影响识别精度的有两个音素：
* 采样率。16k 采样率就足够了，因为大部分语音信号的频率不会高于8k，采样率越低性能越差。
* 位宽。 一般16位。

### 9.3.2 端点检测
端点检测其实就是一个二分类的问题，对于 speech 信号，**可以容忍较高的 FA，必须要有比较低的 FR**，有两种实现方案：
* 基于能量的方案。能量的阈值有一定时间段的语音信号估计得到，并随时间更新。Constraints on word duration can also be imposed to
better classify a sequence of frames so that extremely short spikes can be eliminated.
* 贝叶斯分类器。假设speech 信号与 noise 信号均服从高斯分布\\(\Phi=(\Phi_1, \Phi_2)\\)，由训练数据估计得到分布参数，并在运行过程中采用 EM 算法迭代更新高斯分布参数。
To track the varying background noises, we use an exponential window to give weight to the most recent signal:

$$\omega_k = exp(-\alpha k) $$
where \\(\alpha\\) is a constant that controls the adaptation rate, and \\(k\\) is the index of the time. 实际应用中，对 noise 信号和 speech 信号可以使用不同的 \\(\alpha\\)，前者一般取更小的值。采用上述指数窗，高斯的均值按如下公式进行更新：

$$\hat\mu_k = \frac{\sum_{i=\infty}^{t}\omega_i \frac{c_k P(x_i|\Phi_k)x_i}{\sum_{k=1}^{2}P(x_i|\Phi_k)}}{\sum_{i=\infty}^{t}\omega_i \frac{c_k P(x_i|\Phi_k)}{\sum_{k=1}^{2}P(x_i|\Phi_k)}}, \quad k\in{(1,2)}$$

### 9.3.3 MFCC
详细的 MFCC 特征提取过程可参考 [MFCC 特征提取](https://xingui.github.io/%E8%AF%AD%E9%9F%B3%E8%AF%86%E5%88%AB/2018/05/17/MFCC-%E7%89%B9%E5%BE%81%E6%8F%90%E5%8F%96/#)

### 9.3.4 特征变换
目的：
* 降低计算量--降维(PCA)
* 提高识别率--（LDA，Neural Network, VTLN)


## 9.4 Phonetic Modeling -- selecting appropriate unites
建模单元的选择需要考虑几个因素：
* 准确性。在不同的上下文中能准确的描述。
* 可训练。即拥有足够多的训练数据。
* 泛化性。可以描述新词汇

### 9.4.1 不同建模单元的比较
* Word： 适合小词汇量的系统，例如数字。To summarize, word models are accurate if enough data are available. Thus, they are trainable only for small tasks. They are typically not generalizable.
    1. 优点：通常比其他单元有更好的性能。
    2. 缺点：不能识别训练数据中没有的新词；需要有大量的训练数据
* Phoneme: 
    1. 优点：more trainable and generalizable，即需要更少的训练数据，可以识别训练数据中没有的词
    2. 缺点：phoneme的发音依赖于上下文，过于泛化，降低了模型精度。
* Syllable:
    1. 优点：syllable 的中心部分发音不依赖与上下文，只有边缘部分依赖上下文，是 word 和 phoneme 的折中
    2. 缺点：只适用于部分 syllable 较少的语言，如中文（1200个）和日语（50个），不适用与较多 syllable 的语言，如英语（超过30000个）。
    
### 9.4.2 上下文依赖
* context: 使用建模单元的上下文依赖，可显著提高识别性能。e.g. Triphone
* stress: In most state-of- the-art speech recognition systems, only word-level stress is used for creating allophones.
    1) word level
    2) sentence level
    
### 9.4.3 建模单元聚类
聚类的目的：
* Improve trainability and efficiency。
* Map these unseen triphones into appropriately trained triphones. 提升模型的泛化性能。

两种聚类粒度：
* phonetic
* subphonetic(state)：一般采用**决策树**做Triphone 的状态绑定， 即state-based clustering. 详细步骤可参考：[tree-based state tying for high accuracy acoustic modeling](http://aclweb.org/anthology/H94-1062) 以及 [声学模型之状态绑定](https://xingui.github.io/%E8%AF%AD%E9%9F%B3%E8%AF%86%E5%88%AB/2018/05/31/State_tying_for_acoustic_modeling/#)

Both phonetic and subphonetic units have the same benefits, as they share parameters at unit level. This is the key benefit in comparison to the word units.

性能方面： **Context-independent phone < context-dependent phone < clustered triphone < Senone(clustered state)**

### 9.4.4 Lexical Baseforms
* **作用：** 描述词(word)的发音，构建发音字典
* **如何处理OOV(out of vocabulary):** 使用LTS（Letter-to-Sound）转换器（或称为PG, Pronunciation Guesser）
* **如何构建LTS:** Trainable LTS converters can be based on **neural networks**, **HMMs**, or the **CART**
* **exception dictionary:** 用于添加新词汇的正确发音
* **多音字的处理:** To incorporate widespread pronunciations, we can use a probabilistic finite state machine to model each word’s pronunciation variations. In practice, the relative error reduction of using probabilistic finite state machines is very modest (5–10%).
![](./images/9_2.png)

## 9.5 Acoustic Modeling -- Scoring Acoustic Features

Scoring其实就是计算：\\(P(X | W)\\)

### 9.5.1 Choice of HMM Output Distributions
三种选择：
* Discrete
* Continuous
* Semicontinuous

性能上：**Continuous > Semicontinuous > Discrete**，如下图：   
![](./images/9_3.png)

计算复杂度：**Continuous > Semicontinuous > Discrete**


## 9.6 Adaptive Techniques -- Minimizing Mismatches
One of the most important factors in making a speech system usable is to minimize the possible mismatch
dynamically with a small amount of calibration data. Adaptive techniques can be used to modify system parameters to better match variations in microphone, transmission channel, environment noise, speaker, style, and application contexts.

自适应的方式：
* Transcription independent。取在线识别中 confidence 较高的utt 做 adaptation。
* Transcription dependent。用户按照预先提供的transcription朗读，采用朗读的 utt 做 adaptation。

Challenge：we can use only a small amount of observable data to modify model parameters.

### 9.6.1 Maximum a Posteriori（MAP）
MAP 可以有效的处理数据稀疏的问题，假设 HMM 的模型参数为\\(\mathbf{\Phi}\\)，观测数据为\\(\mathbf{X}\\)，MAP 可表示为下式：

$$\hat{\mathbf{\Phi}} = \arg \max_{\Phi}[P(\mathbf{\Phi}|\mathbf{X})]=\arg \max_{\Phi}[P(\mathbf{X}|\mathbf{\Phi})P(\mathbf{\Phi})]$$

上式的求解通过 EM 算法实现。

缺点：
* 需要有精确的先验知识\\(P(\mathbf{\Phi})\\)
* 只能进行局部优化。只有训练数据相关状态的 HMM 参数才能得到更新。

优点：
* 少量数据就可有较大的性能提升

### 9.6.2 Maximum Likelihood Linear Regression (MLLR)
MLLR 实际做的就是对 GMM 中的高斯均值和协方差做仿射变换，即对于 state i 的第 k 个高斯分量，有

$$\hat{\mathbf{\mu_{ik}}} = \mathbf{A_c{\mu}_{ik}} + \mathbf{b_c}$$

### 9.6.3 MLLR and MAP Comparison
* MLLR 和 MAP 同时做，可以获得更好的性能
* 少量训练数据时，MLLR 有更好的性能，随着数据量的增加，MAP 的性能会逐渐好于 MLLR
![](./images/9_4.png)


## 9.7 Confidence measures: Measuring the reliability
confidence 的计算可以表示为：

$$P(\mathbf{W}|\mathbf{X})=\frac{P(\mathbf{W})P(\mathbf{X}|\mathbf{W})}{P(\mathbf{X})}=\frac{P(\mathbf{W})P(\mathbf{X}|\mathbf{W})}{\sum_W P(\mathbf{W})P(\mathbf{X}|\mathbf{W})}$$

### 9.7.1 Filler Models
上式中，分子部分\\(P(\mathbf{W})P(\mathbf{X}|\mathbf{W})\\)一般通过维特比解码得到，分母部分一般通过 Filler Model 计算得到。**One of the most widely used is the socalled all-phone network, in which all the possible phonetic and nonspeech HMMs are connected to each other, and with which any word sequence can be recognized.**

In practice, we don’t need to use all the hypotheses to compute \\(\sum_W P(\mathbf{W})P(\mathbf{X}|\mathbf{W})\\), Instead, n-best lists and scores an be used to provide an effective
estimate of \\(\sum_W P(\mathbf{W})P(\mathbf{X}|\mathbf{W})\\)。

### 9.7.2 Transformation Models
对一个 word 中不同 phoneme 的 confidence 加权求和，得到 word 的 confidence。If a word \\(w\\) has \\(N\\) phones, we can compute the confidence score of the word as follows:

$$CS(w) = \frac{\sum_{i=1}^{N}\phi_i(x_i)}{N}$$

其中 \\(CS(w)\\)为 word confidence， \\(x_i\\)是phoneme 的 confidence，\\(\phi_i\\)是变换函数，一般做仿射变换，即

$$\phi_i(x) = ax + b$$

变换参数由区分性训练得到。

### 9.7.3 Combination Models
In practical systems, there are a number of features you can use to improve the performance of confidence measures. 
* Word stability ratio from different language model weights (WrdStabRatio). This is obtained by applying different language weights to see how stably each word shows up in the recognition n-best list.
* Logarithm of the average number of active words around the ending frame of the word (WrdCntEnd).
* Acoustic score per frame within the word normalized by the corresponding active senone scores (AscoreSen).
* Logarithm of the average number of active words within the word (WrdCntW).
* Acoustic score per frame within the word normalized by the phonetic filler model (AscoreFiller).
* Language model score (LMScore).
* Language model back-off (trigram, bigram, or unigram hit) for the word(LMBackOff).
* Logarithm of the average number of active states within the word (StateCnt).
* Number of phones in the word (Nphones).
* Logarithm of the average number of active words around the beginning frame of the word (WrdCntBeg).
* Whether the word has multiple pronunciations (Mpron).
* Word duration (WordDur).

## 9.8. Other Techniques
* **Neural Network**
    1. recurrent network（RNN）
    2. Time Delay Neural Network（**TDNN**)
* **Segment Models**

